In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from utils import *
import pickle
import pandas as pd

# Carregando o modelo de clusterização e o de classificação

In [2]:
with open('modelos/kmeans.pkl', 'rb') as f:
    kmeans = pickle.load(f)

with open('modelos/pipeline.pkl', 'rb') as f:
    pipeline = pickle.load(f)

no_clusters = 200

In [3]:
def calcular_pontos_retangulo(ponto1, ponto2):
    x1, y1 = ponto1
    x2, y2 = ponto2
    
    return [y1, y2, x1, x2]

# Definições das vagas

In [4]:
video = 'curto'
offset = 0 if video == 'curto' else 70

vagas = np.array([
    [(79, 394 + offset), (288, 667 + offset)],
    [(526, 394 + offset),(694, 667 + offset)],
    [(727, 394 + offset), (979, 667 + offset)],
    [(1001, 394 + offset), (1176, 667 + offset)],
    [(1215, 394 + offset), (1378, 667 + offset)],
    [(1422, 394 + offset), (1591, 667 + offset)]
], np.int32)

crop = [None for _ in vagas]
descriptores = [None for _ in vagas]
features = [None for _ in vagas]

sift = cv2.SIFT_create()


# Aplicação do método em tempo real

In [8]:
video_path = f"videos/{video}.mp4"

# Função de callback para capturar o clique do mouse
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  
        print(f"Coordenadas: ({x}, {y})")

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo")
else:
    cv2.namedWindow('Video')
    cv2.setMouseCallback('Video', mouse_callback)
    fps = cap.get(cv2.CAP_PROP_FPS)

    tempo_ocupado = [0 for _ in vagas]
    frames_ocupados = [0 for _ in vagas]
    ocupacao = [0 for _ in vagas]
    
    contador = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            novo_frame = frame.copy()

            if contador == 0:
                for i, vaga in enumerate(vagas):
                    p = calcular_pontos_retangulo(vaga[0], vaga[1])
                    crop[i] = frame[p[0]:p[1], p[2]:p[3]]

                    img = readImage(crop[i])
                    des = getDescriptors(sift, img)

                    if des is not None:
                        descriptores[i] = des
                    else:
                        descriptores[i] = np.zeros((1, sift.descriptorSize()), np.float32)

                    #cv2.imshow(f'crop_{i}', crop[i])

                novo_im_features = extractFeatures(kmeans, descriptores, no_clusters)
                pred = pipeline.predict(novo_im_features)

            if contador < 60:
                contador += 1
            else:
                contador = 0
            
            for i, vaga in enumerate(vagas):
                if pred[i] == 1:
                    frames_ocupados[i] += 1
                    ocupacao[i] = 1
                else:
                    ocupacao[i] = 0
                
                tempo_ocupado[i] = frames_ocupados[i] // fps

                cv2.rectangle(novo_frame, vaga[0], vaga[1], color=(0, 255, 0) if not ocupacao[i] else (0, 0, 255), thickness=2)

                novo_frame = cv2.putText(novo_frame, f'{int(tempo_ocupado[i] // 60):02}:{int(tempo_ocupado[i] % 60):02}', 
                                         ponto_central_retangulo(vaga[0], vaga[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                                         1, (0, 255, 0) if not ocupacao[i] else (0, 0, 255), 2, cv2.LINE_AA)
            
            novo_frame = cv2.putText(novo_frame,f'Total de vagas: {len(vagas)}', (1566, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            novo_frame = cv2.putText(novo_frame,f'Vagas ocupadas: {np.sum(ocupacao)}', (1566, 90), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) 
            novo_frame = cv2.putText(novo_frame,f'Ocupacao: {np.sum(ocupacao) / len(vagas) * 100:.2f}%', (1566, 130), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) 

            cv2.imshow('Video', novo_frame)

            # Pressione 'q' para sair do loop e fechar a janela
            if cv2.waitKey(3) & 0xFF == ord('q'):
                break
        else:
            break

# Libera o objeto de captura e fecha todas as janelas abertas
cap.release()
cv2.destroyAllWindows()


# Aplica o método e salva o vídeo resultante

In [11]:
video_path = f"videos/{video}.mp4" 
output_path = f"videos/resultado_{video}.avi"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo")
else:

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)

    fourcc =  cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    tempo_ocupado = [0 for _ in vagas]
    frames_ocupados = [0 for _ in vagas]
    ocupacao = [0 for _ in vagas]
    indices = []
    serie_temporal = []

    contador = 0
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret:

            novo_frame = frame.copy()

            if contador == 0:
                for i, vaga in enumerate(vagas):
                    p = calcular_pontos_retangulo(vaga[0], vaga[1])
                    crop[i] = frame[p[0]:p[1], p[2]:p[3]]

                    img = readImage(crop[i])
                    des = getDescriptors(sift, img)

                    if des is not None:
                        descriptores[i] = des
                    else:
                        descriptores[i] = np.zeros((1, sift.descriptorSize()), np.float32)

                novo_im_features = extractFeatures(kmeans, descriptores, no_clusters)
                pred = pipeline.predict(novo_im_features)

            if contador < 15:
                contador += 1
            else:
                contador = 0

            for i, vaga in enumerate(vagas):
                if pred[i] == 1:
                    frames_ocupados[i] += 1
                    ocupacao[i] = 1
                else:
                    ocupacao[i] = 0
                
                tempo_ocupado[i] = frames_ocupados[i] // fps

                cv2.rectangle(novo_frame, vaga[0], vaga[1], color=(0, 255, 0) if not ocupacao[i] else (0, 0, 255), thickness=2)

                novo_frame = cv2.putText(novo_frame, f'{int(tempo_ocupado[i] // 60):02}:{int(tempo_ocupado[i] % 60):02}', 
                                         ponto_central_retangulo(vaga[0], vaga[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                                         1, (0, 255, 0) if not ocupacao[i] else (0, 0, 255), 2, cv2.LINE_AA)
            
            novo_frame = cv2.putText(novo_frame,f'Total de vagas: {len(vagas)}', (1566, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            novo_frame = cv2.putText(novo_frame,f'Vagas ocupadas: {np.sum(ocupacao)}', (1566, 90), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) 
            novo_frame = cv2.putText(novo_frame,f'Ocupacao: {np.sum(ocupacao) / len(vagas) * 100:.2f}%', (1566, 130), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            indices.append(cap.get(cv2.CAP_PROP_POS_FRAMES) / fps)
            serie_temporal.append(ocupacao.copy())

            out.write(novo_frame)

        else:
            print(cap.get(cv2.CAP_PROP_POS_FRAMES) / cap.get(cv2.CAP_PROP_FRAME_COUNT))
            break

cap.release()
out.release()

29.97002997002997
1.0


# Exporta dados csv

In [34]:
pd.DataFrame(serie_temporal, index=indices).to_csv(f'dados/{video}_BOF.csv')